In [3]:
def get_genome_file(genome_location,species):
    list_of_files_in_genome_folder = os.listdir(f"{genome_location}/{species}")
    for file in list_of_files_in_genome_folder:
        if file.endswith("_genomic.fna"):
            genome_file = file
    return(genome_file)  

def get_gene_sequence(genome_location, 
                      species, 
                      genome_file, 
                      scaffold,
                      gene_start,
                      gene_end,
                      complement,
                      output_location):

    
    print("Getting Gene")
    list_of_folders = os.listdir(output_location)
    # print(list_of_files_inside_annotated_species_folder)
    # print(
    if f"0.Temp" not in list_of_folders:
        os.mkdir(f"{output_location}/0.Temp")

#     subprocess.run(f'samtools faidx "{genome_location}/{species}/{genome_file}"', shell = True, stderr = subprocess.DEVNULL)
#     print(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{gene_start}-{gene_end}')
    subprocess.run(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{gene_start}-{gene_end} > "{output_location}/0.Temp/temp_genome.fa"', shell = True, stderr = subprocess.DEVNULL)
    
    genome = SeqIO.parse(f"{output_location}/0.Temp/temp_genome.fa", "fasta")
    for entries in genome:
        gene_sequence = entries.seq
        if complement == "1":
            gene_sequence = gene_sequence.reverse_complement()
        break
    # print(gene_sequence)
    return (gene_sequence)
def make_output_folder(output_location):
    list_of_folders = os.listdir(output_location)
    if "2.Final_output" in list_of_folders:
        subprocess.run(f'rm -r "{output_location}/2.Final_output"', shell = True, stderr = subprocess.DEVNULL)
    os.mkdir(f"{output_location}/2.Final_output")

def write_exon_output(output_location,species,exon,exon_output):
    
    
    list_of_folders_in_final_output = os.listdir(f"{output_location}/2.Final_output")
    if species not in list_of_folders_in_final_output:
        os.mkdir(f"{output_location}/2.Final_output/{species}")
    
    with open(f"{output_location}/2.Final_output/{species}/{exon}.fa", 'w') as out_file:
        out_file.write(exon_output)

In [7]:
import os
from Bio import SeqIO
import subprocess

family_group = "3.Satyrine"

output_location = f"/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/{family_group}"
make_output_folder(output_location)
genome_location = "/mnt/f/Genomes_2023-12-26"
list_of_species = os.listdir(f"{output_location}/1.Blast_result")
if 'desktop.ini' in list_of_species:
    list_of_species.remove("desktop.ini")
# print(list_of_species)
# list_of_species = ["Heliconius_charithonia"]
# list_of_species = ["Vanessa_tameamea"]
error_exons = ''
for species in list_of_species:
    print(species)
    genome_file = get_genome_file(genome_location,species)
    print(genome_file)
    previous_coordinate_start = ''
    previous_scaffold = ''
    with open(f"{output_location}/1.Blast_result/{species}/final_coordinates.csv", 'r') as coordinate_final_file:
        coordinate_file_list = coordinate_final_file.readlines()
    
    for i in range(1,len(coordinate_file_list)):
        
        coordinate_line_split = coordinate_file_list[i].split(",")
        
            
#         print(coodinate_line_split[5])
        if coordinate_line_split[5] == "N":
            if previous_coordinate_start == '':
                previous_coordinate_start = coordinate_line_split[2]
                previous_scaffold = coordinate_line_split[1]
            complement = coordinate_line_split[4]
            start,stop = coordinate_line_split[2],coordinate_line_split[3]
            print(start,stop)
            query_name = coordinate_line_split[6]
            scaffold = coordinate_line_split[1]
            exon = f"Exon_{query_name.split('_')[-1]}"
            left_oh, right_oh = query_name.split('_')[-6], query_name.split('_')[-4]
#             print(complement)
            if complement == "0":
                if start < previous_coordinate_start or scaffold != previous_scaffold:
                    print(f"Error in Coordinates\nPrevious_coordinate = {coordinate_file_list[i-1]}\nCurrent_coordinate = {coordinate_file_list[i]}")
                    prompt = input("Continue (Y/N)?")
                    if prompt[0].lower() == "n":
                        assert False
            elif complement == "1":
                if start > previous_coordinate_start or scaffold != previous_scaffold:
                    print(f"Error in Coordinates\nPrevious_coordinate = {coordinate_file_list[i-1]}\nCurrent_coordinate = {coordinate_file_list[i]}")
                    prompt = input("Continue (Y/N)?")
                    if prompt[0].lower() == "n":
                        assert False
            else:
                print("ERROR")
            
        elif coordinate_line_split[5] == "Y":
            error_exons += coordinate_file_list[i]
        
        gene_sequence = get_gene_sequence(genome_location, 
                                          species, 
                                          genome_file, 
                                          scaffold,
                                          start,
                                          stop,
                                          complement,
                                          output_location)
        
       
        current_exon = query_name.split('_')[-1]
        if current_exon == "1":
            if complement == 1:
                translated = gene_sequence.reverse_complement.translate()
            else:
                translated = gene_sequence.translate()
            print(translated)
            if "*" in translated:
                error_exons += f"{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n"
            else:
                exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
                print(exon_output)
        
                write_exon_output(output_location,species,exon,exon_output)
        elif current_exon == "24":
            if complement == 1:
                translated = gene_sequence.reverse_complement.translate()
            else:
                translated = gene_sequence.translate()

            if "*" in translated[:-1]:
                error_exons += f"{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n"
                print(translated)
            else:
                
                exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
                print(exon_output)
                write_exon_output(output_location,species,exon,exon_output)
#         assert False
        else:
            exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
            print(exon_output)

            write_exon_output(output_location,species,exon,exon_output)
        
#         assert False
        previous_coordinate_start = start
        previous_scaffold = scaffold
    error_exons += "\n\n"
#     print(coordinate_file_list)
#     break
coordinate_file = ''
print(error_exons)
with open(f"{output_location}/error_exons.txt", 'w') as error_out_file:
    error_out_file.write(error_exons)

Bicyclus_anynana
GCF_947172395.1_ilBicAnyn1.1_genomic.fna
7826744 7826832
Getting Gene
MDNMDDSDPNAKISDSAYSTSCSNSQSRR
>Bicyclus_anynana_Exon_1_NC_069110.1_7826744_7826832_left_0_right_2
ATGGACAACATGGACGATTCTGACCCTAATGCGAAAATATCGGACTCCGCTTATTCTACAAGCTGTAGTAATAGCCAATCTAGAAGAAG
7833163 7833234
Getting Gene
>Bicyclus_anynana_Exon_2_NC_069110.1_7833163_7833234_left_1_right_2
TCACAGTTCGAAATCTACGCATTCAAGCGGAAGCAGTGGATATGGTGGGAAGCAGACGACTTCTGGATACAA
7833419 7833594
Getting Gene
>Bicyclus_anynana_Exon_3_NC_069110.1_7833419_7833594_left_1_right_1
cAACAACAATCAAACCCAGTCACCGGAAAAAAGACCCAAGGAAAAGGAATTGAAGAAAAAGAAGACTCAGCTAACTACACAAAATGATGTACAAGTTGTTGAGgctaaaattgaaattgtatgTGATGTGACAATGAAACCGGTCTTTGAACCATTGCCGGAAGAAATTAGCGATC
7835512 7835597
Getting Gene
>Bicyclus_anynana_Exon_4_NC_069110.1_7835512_7835597_left_2_right_0
TTGTTCCAAGCCCAGCATCCGGCGTGGTGAAGATGGAGAAGGGACCTGAGTTGGATGTATCTGATAAAAAAAGTGGAAAAGACGAG
7835941 7836009
Getting Gene
>Bicyclus_anynana_Exon_5_NC_069110.1_7835941_7836009_left_0_right_0


7038094 7038205
Getting Gene
>Coenonympha_glycerion_Exon_10_OY979626.1_7038094_7038205_left_1_right_0
GATGATGGTACAAAATGGAGATTACTTGAAGCTAGAGACTGAATGGTCCTCTTTTACAAATCCGTGGTCCAGAAAACTTGAGTTTGTAATTGGGAAACATTACATATTAGAG
7038287 7038406
Getting Gene
>Coenonympha_glycerion_Exon_11_OY979626.1_7038287_7038406_left_0_right_0
GGTCCAACAAATCCTGACGTTTTCCAGACACCAGAACCTGAAAAAGTAACTTACGCATCTAAAGAGGAAGAAGCCAAAGCTCAGTTAATACGAGAACATATGGTCCGAATTATGAATGag
7038845 7038994
Getting Gene
>Coenonympha_glycerion_Exon_12_OY979626.1_7038845_7038994_left_0_right_0
GTACTAACAAAACCAGCGGAAGTAGCTAAACAGCAGATGAGCAAACGCTGTCAGGACTTAGCGACGTTTGTGGAAAGCTTGATGGATGACACTCCAGTGCCTGAAGACGAACTACGCGTGGAAATACAAGACGCGGATTACAGTTATTTT
7039404 7039522
Getting Gene
>Coenonympha_glycerion_Exon_13_OY979626.1_7039404_7039522_left_0_right_2
GAGCATGATTCGGTGTTGTTAGGCGGTATTTCTCCTCACCACGATTACAATGATAGTGACACGAGCACTGAGACGCCCCTGAGCTATAACCAACTCAATTATAATGAAAATCTTCAAAG
7040150 7040251
Getting Gene
>Coenonympha_glycerion_Exon_14_OY979626.1_7040150_7040251_

>Erebia_aethiops_Exon_18_OV281079.1_7638571_7638675_left_2_right_1
TACCGTCACCATGGCCATCCAATCAGGGCAACAATATGAATGCATTTATTCTTGGCTTGGGTATTCCGCCCCAGATGTCTGTTATGAGCCCCATGGCGATGCCTG
7639269 7639370
Getting Gene
>Erebia_aethiops_Exon_19_OV281079.1_7639269_7639370_left_2_right_1
GTGTCTTACCGGTGTATTATACTCCGATGACGAACCAGCAAATGCCACCTAGATTCAACACAAATAGTGCTATGCCCCATGCACATTATTATAGATATCCAA
7639930 7640106
Getting Gene
>Erebia_aethiops_Exon_20_OV281079.1_7639930_7640106_left_2_right_1
GTGTAAACGTTCAGACTTTAAATCCCCATATTCAATATATGCAACAAGGACAAATGAATGTACCTGACGTGGTATTTCAAATGAACTCTGTATTTTCTACAAACGGGGAACCGCACAGTGTCAACCCTAATATTGTGCCACATAATCAGTATCAACAGCAACATCAAAATAATCAat
7640747 7640901
Getting Gene
>Erebia_aethiops_Exon_21_OV281079.1_7640747_7640901_left_2_right_0
TACAATGTATGATGCTGGGTCAGACAGTCTACGGACCGCCGTTCATGTACGCCCCCATAGATCCGCAAATGTCATACGCATTACAACCAAGTTTTATACCACAGCAGACGCTTAATACGCCACCATTGGGCTTTTCAAGTAGTACCTACGAAGAG
7641500 7641613
Getting Gene
>Erebia_aethiops_Exon_22_OV281079.1_7641500_7641613_left_0_right_0
gctact

>Erebia_ligea_Exon_27_OU785222.1_13264575_13264703_left_0_right_0
AGTCGCCGGCGAAAGCGTGAGTACAGCAGATTAGTAATGATCTACGAAGAAGATGCTCCATTTCCTCCACCAGATGATACTAGCAGTTCTTCACTTTTTAGTATTATCGCGGAGAGTCAGGGCACGTGA
Hipparchia_semele
GCA_933228805.2_ilHipSeme1.2_genomic.fna
513966 514054
Getting Gene
MDNLDDSENNAKISDSAYSNSCSNSQSRR
>Hipparchia_semele_Exon_1_OW121737.1_513966_514054_left_0_right_2
ATGGACAACCTGGACGATTCAGAGAATAATGCGAAAATATCGGATTCCGCTTATTCTAACAGCTGCAGCAACAGTCAATCTAGAAGAAG
512237 512320
Getting Gene
>Hipparchia_semele_Exon_2_OW121737.1_512237_512320_left_1_right_2
TCATAGTTCAAAGTCTACACGTTCTGGTAGCAACTCAAGTGGAAGCAGTGGATATGGTGGAAAACCGTCTACTTCTGGATACAG
511716 511891
Getting Gene
>Hipparchia_semele_Exon_3_OW121737.1_511716_511891_left_1_right_1
CAGTAACAATCAAAGTCAACCAACTGAAAAAAGATCCAAAGATAAAGACGCGAAGAAGAAAAAGCCTTTACAGAATATCCAAACTGATTTACAAGGTGCCGAGGTTAAAAATGTAATCGAATGCATACCGGAAATTGTGCCAGTATTTGAATCATCAAAAGAAGAAATAAATGATG
511012 511100
Getting Gene
>Hipparchia_semele_Exon_4_OW121737.1_511012_511100_left_2

>Lasiommata_megera_Exon_10_OV743307.1_17357754_17357865_left_1_right_0
gatGATAACGCAAAACGGAGACTACTTGAAACTAGAGACAGAATGGTCATCATTCACTAATCCATGGTCCAGAAAGCTAGAGTTTGTAATCGggaaacattttataattgag
17357003 17357122
Getting Gene
>Lasiommata_megera_Exon_11_OV743307.1_17357003_17357122_left_0_right_0
GGCCCGAGAAATCCTGATGTTTTCCAGACGCCGGAGGttgaaaaaagttttaaaataactaatgaGGACGAGGCAAAGGCTCAGGCTTTAAGAGAAAGCATGGTCAGAATTATGAATGAG
17356697 17356849
Getting Gene
>Lasiommata_megera_Exon_12_OV743307.1_17356697_17356849_left_0_right_0
GTGCTGACTAAACCTGCGGAAATAGCGAAGCAGCAGATGACTAAACGCTGTCAGAACTTAGCATCCTTTATGGAAAGTTTATTGGAGACTACACCTCCTAAACAAGAAGAAGAACTCCGCCTAGACATACAAGACGCAGACCAGAGCTGCTAT
17355710 17355828
Getting Gene
>Lasiommata_megera_Exon_13_OV743307.1_17355710_17355828_left_0_right_2
GAGCGCGACTCGGTGATGCTAGGCGGCATTTCCCCGCATCATGACTTCAACGACTGTAACACAAGCACTGAGACTCCATTGACATACAACCAACTTAATTACAATGAAAATCTACAAAG
17354503 17354607
Getting Gene
>Lasiommata_megera_Exon_14_OV743307.1_17354503_17354607_left_1_right_2
ATTCTTTGATTGC

6333729 6333899
Getting Gene
>Maniola_hyperantus_Exon_20_NC_048564.1_6333729_6333899_left_2_right_1
ATGCAAGCACTTCAAATCCTCATATGCAGTATGTGCAACAAGGACAAATTAATGAGTCTGACGTGGTGTTTCAAATGAACTCTGTGTTTTCCACGAGTGGGGAAGCAAACAGAGTCAACCCTCTTGATATCCCACCTATTCAGTATCAGCAAGGACATCAAAGTAGTCAAA
6334660 6334814
Getting Gene
>Maniola_hyperantus_Exon_21_NC_048564.1_6334660_6334814_left_2_right_0
TGCAATGTATGATGCTGGGTCAGGCAGTCTATGGATCTCCGTACATGTACTCAGCTGTAAATCAGCAGATGCCCTACGCAGTGCAACAAAGATTTATACCACAACAAACACCTCATACGCAACCATTGGAATTGTCAAGTAGTAACTATGAAGAG
Getting Gene
>Maniola_hyperantus_Exon_21_NC_048564.1_6334660_6334814_left_2_right_0
TGCAATGTATGATGCTGGGTCAGGCAGTCTATGGATCTCCGTACATGTACTCAGCTGTAAATCAGCAGATGCCCTACGCAGTGCAACAAAGATTTATACCACAACAAACACCTCATACGCAACCATTGGAATTGTCAAGTAGTAACTATGAAGAG
6338233 6338406
Getting Gene
>Maniola_hyperantus_Exon_23_NC_048564.1_6338233_6338406_left_0_right_0
AGTACCGATACTGTGACTGATGAACGTATGAGAAGGCTTGGCAATAGTGACAGTGCTAACGATAAGACTGATGGAGAGTCGAGCTACTCGTCTTTCTATTCATCTTTCTTTAAGACTGACTCTGGAAGTGGC

MDTLDDSENTAKISDSAYSNSCSNSQSRR
>Melanargia_galathea_Exon_1_OV049860.1_10020919_10021007_left_0_right_2
ATGGATACCCTGGACGATTCAGAGAATactgcgaaaatttcggattccGCTTATTCTAACAGTTGCAGCAACAGCCAGTCTAGAAGAAG
10022153 10022236
Getting Gene
>Melanargia_galathea_Exon_2_OV049860.1_10022153_10022236_left_1_right_2
TCATAGCTCAAAATCTACACGTTCTGGTAGCAATTCAAGCGGGAGCAGTGGCTATGGTGGAAAGCCGTCTACTTCTGGATACAG
10027781 10027956
Getting Gene
>Melanargia_galathea_Exon_3_OV049860.1_10027781_10027956_left_1_right_1
CAGCAACATTCAGATTGAACCAACAGAAAAAAGAACAAAAGACAAAGAAATAAAGAAGAAAAAGCCTTTGCAGAGTCTCCAAACAGATTTACAAGTTACCGAGGCtaaaaatgaaattgaatgcaTTCCGGAAAACGTGCCGATCTTTGAATCATCTAAGGAAGATATCAATGATG
10029176 10029264
Getting Gene
>Melanargia_galathea_Exon_4_OV049860.1_10029176_10029264_left_2_right_0
TCGCTCCAATACCAACGCCCGTACTGGCGCAGACTGAGAAGGTGCCTGACTTGATGGAAGAGACTGATAAGAGCAATGGAAAGGACGAG
10030097 10030171
Getting Gene
>Melanargia_galathea_Exon_5_OV049860.1_10030097_10030171_left_0_right_0
CCCGTTGTGAGCAACCCGCGGGTAGAGACTCCCATAACATTTGT

13923817 13923928
Getting Gene
>Oeneis_ivallda_Exon_10_JARPMR010000001.1_13923817_13923928_left_1_right_0
AATGATGGCGCAAAATGGAGATTACTTGAAGCTGGAGACTGAATGGACGTCTTTCATCAATCCATGGTCACGGAAGCTTGAATTCGTAATTGGAAAACATTTCATACTACAG
13924022 13924141
Getting Gene
>Oeneis_ivallda_Exon_11_JARPMR010000001.1_13924022_13924141_left_0_right_0
GGCCCTACAAATCCCGACGTTTTCCAGACCCCGGATCCCGAAAAAGCTTATAAAAAGCCTTCCGAAGAAGAAACAAAGAGCCAAATAATAAGAGACAATATCATCAGAATTATGAATGAG
13925258 13925407
Getting Gene
>Oeneis_ivallda_Exon_12_JARPMR010000001.1_13925258_13925407_left_0_right_0
GTACTGACTAAGCCTGCGGAAGTAGCTAAACAGCAGATGAGTAAACGGTACCAGGATCTGGCATCTTTTATGGAAAGTTTGCTGGAAGAAATGCCTAAACCTGTAGAGGAACTTCGCCTAGAAATACAAGATGCAGATCACAGCTGTTTT
13926509 13926627
Getting Gene
>Oeneis_ivallda_Exon_13_JARPMR010000001.1_13926509_13926627_left_0_right_2
GAGCGTGATTCAGTGATGCTAGGTGGTATATCCCCTCATCACGAATACAATGACAGTAACACAAGCACAGAGACAACCCTGAGCTACAATCAACTTAACTACAAAGAAAATCTACAAAG
13926750 13926842
Getting Gene
>Oeneis_ivallda_Exon_14_JARPMR010000001.

16915201 16915290
Getting Gene
>Pararge_aegeria_Exon_19_NC_053208.1_16915201_16915290_left_2_right_1
GTCTCTTACCGGTAGACTATGCTCCACCAATGCCCTATCAAGTGCCTCCAAGATCTGATTTAGCTGCTCCTCATTATGATAGGCATTCAA
16911822 16911965
Getting Gene
>Pararge_aegeria_Exon_20_NC_053208.1_16911822_16911965_left_2_right_1
ATATGGCGCCCGTAAATCCGCAAATGCAGTACCACCATATGCCCCAACAAATGAACCCACATTGTTCTACTGGTGCTGTTCCGAATAGGTACATGCACAATAACATGCAACCTGCTTATCCAGTTCATGTACCAAATCCTCAaa
16910459 16910610
Getting Gene
>Pararge_aegeria_Exon_21_NC_053208.1_16910459_16910610_left_2_right_0
tgcctTGCATGATGATGGGTCAGCCATTCTATGGAGGGTTTTATGTGTACCCTCAAATGGAACCCCGAGTACCGTATGTCCACCAAGGTTACATACAACAGCAAATGCCGAATACCCAGCCATTAGGACTGTCCAATAGAAATTATGAAGAG
16908496 16908624
Getting Gene
>Pararge_aegeria_Exon_22_NC_053208.1_16908496_16908624_left_0_right_0
GCATGTAAAGTAACAATGCCACCCAAGGCTGGTACATCTGGTAATGGAAGTAAGAGGCGGGAGAAGATAACTGAGtatcgagaaaaaaaaagtgacacCGGTGGCTCTAACGTCGGTGCCTCAAATCGG
16904903 16905082
Getting Gene
>Pararge_aegeria_Exon_23_NC_053208.1_16904903_